### Import library

In [1]:
import os
os.chdir('../..')
os.getcwd()

'C:\\Users\\csia7\\OneDrive\\문서\\GitHub\\WQBrain_2024_API'

In [2]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [3]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_46eCs2ZvxkqGRNtfKB6nCZZpKR8X

 


In [5]:
ace.check_session_timeout(s)

14398.65292

## Alpha Mixing Templates

## Read all passed alphas

In [4]:
os.chdir('Results/signal')
file_direc = glob.glob('*')
file_direc

['A_signalform.csv',
 'B_signalform.csv',
 'C_signalform.csv',
 'D_signalform.csv',
 'pre_signalform.csv',
 'Sep03GLB_signalform.csv']

In [5]:
expressions = {}
for file in file_direc:
    expressions[file.split('_')[0]] = pd.read_csv(file)['expression']
#expressions

In [6]:
os.chdir('../..')

In [7]:
expressions.keys()

dict_keys(['A', 'B', 'C', 'D', 'pre', 'Sep03GLB'])

In [8]:
expressions['pre']

0    sig_pre_0 = group_zscore(ts_zscore(ts_backfill...
Name: expression, dtype: object

## Template A & pre mixing
use top 32 results

In [59]:
#region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01

In [9]:
B = expressions['B']
pre = expressions['pre']
#Sep03GLB = expressions['Sep03GLB']

In [10]:
alpha_list_Bpre = []

for i in range(20):
    alpha_list_Bpre += [ace.generate_alpha(f'{B[i]};{pre[0]};add(scale_down(signal_B_{i}), scale_down(signal_pre_{0}), filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = x, truncation = 0.01, decay = 5) for x in ['COUNTRY', 'SECTOR']]

alpha_list_Bpre

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 5,
   'neutralization': 'COUNTRY',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'entry_B_0 = ts_rank( ts_std_dev(oth460_3l_vlc, 21), 252 ); maintenance_B_0 = -ts_regression(rsk70_mfm2_gemtrd_srtindcnt, ts_delay(rsk70_mfm2_gemtrd_anlystsn, 1), 252); quit_B_0 = entry_B_0 < 0.5;signal_B_0 =  trade_when(entry_B_0>0.55, maintenance_B_0, quit_B_0);sig_pre_0 = group_zscore(ts_zscore(ts_backfill(anl69_epss_best_eeps_cur_yr, 21), 21), industry);factor_pre_0 = group_zscore(ts_ir(returns, 252), industry);value_pre_0 = floor(rank(fnd28_value_05480/close)*4.99);signal_pre_0 = group_neutralize(sig_pre_0, value_pre_0 *factor_pre_0);add(scale_down(signal_B_0), scale_down(signal_pre_0), filter = false)'},
 {'type':

In [11]:
result = ace.simulate_alpha_list_multi(s, alpha_list_Bpre)

100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [46:10<00:00, 197.92s/it]


In [20]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,25815099,20000000,647,611,0.5515,0.2493,0.0533,0.000904,2.65,3.94,2012-01-22,7weR901,entry_B_8 = ts_rank( ts_std_dev(oth460_veia_l2...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
1,25468835,20000000,634,599,0.5537,0.2459,0.0511,0.000888,2.62,3.93,2012-01-22,Lp0YPva,entry_B_3 = ts_rank( ts_std_dev(oth460_veia_l2...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
2,19814618,20000000,629,596,0.5500,0.1913,0.0303,0.000696,2.53,4.29,2012-01-22,Ppkmde7,entry_B_6 = ts_rank( ts_std_dev(oth460_1l_2dls...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
3,19826093,20000000,644,611,0.5560,0.1914,0.0347,0.000689,2.50,4.26,2012-01-22,8zMx1kW,entry_B_8 = ts_rank( ts_std_dev(oth460_veia_l2...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
4,19577723,20000000,633,600,0.5578,0.1890,0.0335,0.000678,2.46,4.22,2012-01-22,eOXZlZz,entry_B_3 = ts_rank( ts_std_dev(oth460_veia_l2...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
5,24302236,20000000,632,595,0.5452,0.2347,0.0463,0.000861,2.45,3.73,2012-01-22,273obEY,entry_B_6 = ts_rank( ts_std_dev(oth460_1l_2dls...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
6,24367866,20000000,685,648,0.6170,0.2353,0.0503,0.000763,2.40,3.89,2012-01-22,KbWMjRx,"entry_B_4 = ts_rank( ts_std_dev(oth460_3l_vlc,...",PASS,PASS,PASS,PASS,PASS,PASS,PASS
7,24133568,20000000,672,635,0.6206,0.2330,0.0508,0.000751,2.38,3.89,2012-01-22,qAQJMKA,"entry_B_0 = ts_rank( ts_std_dev(oth460_3l_vlc,...",PASS,PASS,PASS,PASS,PASS,PASS,PASS
8,23230706,20000000,612,579,0.5600,0.2243,0.0453,0.000801,2.36,3.73,2012-01-22,6w5Ln6E,entry_B_1 = ts_rank( ts_std_dev(oth460_1l_1dls...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
9,19412765,20000000,683,648,0.6248,0.1875,0.0341,0.000600,2.35,4.29,2012-01-22,Wpv2EZo,"entry_B_4 = ts_rank( ts_std_dev(oth460_3l_vlc,...",PASS,PASS,PASS,PASS,PASS,PASS,PASS


In [22]:
path = 'Results/mixing'
result_st1.to_csv(path + '/B&pre_mixing_add.csv')

In [23]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,date,endDate,limit,name,result,startDate,themes,value,year
0,6w5Ln6E,NaN,NaN,1.58,LOW_SHARPE,PASS,NaN,NaN,3.73,NaN
1,6w5Ln6E,NaN,NaN,1.00,LOW_FITNESS,PASS,NaN,NaN,2.36,NaN
2,6w5Ln6E,NaN,NaN,0.01,LOW_TURNOVER,PASS,NaN,NaN,0.56,NaN
3,6w5Ln6E,NaN,NaN,0.70,HIGH_TURNOVER,PASS,NaN,NaN,0.56,NaN
4,6w5Ln6E,NaN,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,NaN


In [24]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['QawpmEQ' 'jZqKNg5']
Passed alphas:['8zMx1kW', 'lVoPLnn', 'qAQJMKA', 'KbWMjRx', 'eOXZlZz', 'Ppkmde7', 'ad59v2O', 'k3z6qqk', 'Lp0YPva', '9wNOzNr', '7weR901', '6w5Ln6E', 'QawpLe5', '17rAQ6m', '273obEY', 'xxJpKLg', 'Wpv2EZo', 'm8EWMk1']


In [25]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['Sep07_mixing'])

In [27]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas[4:6]}
submit_result

{'eOXZlZz': False, 'Ppkmde7': True}

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

In [37]:
ace.get_alpha_pnl(s, result[0]['alpha_id'])

,Pnl,alpha_id
Date,,
2012-01-23,0.0,dx9ALqJ
2012-01-24,52190.0,dx9ALqJ
2012-01-25,32038.0,dx9ALqJ
2012-01-26,37403.0,dx9ALqJ
2012-01-27,53631.0,dx9ALqJ
...,...,...
2022-01-17,12935472.0,dx9ALqJ
2022-01-18,12935661.0,dx9ALqJ
2022-01-19,12967474.0,dx9ALqJ
